<a href="https://colab.research.google.com/github/ChoiSenn/Study/blob/main/data_analysis/DACON/DACON_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **데이콘 - 중고차 가격 예측 경진대회**

<br><br>

## XGBoost
> XGBoost를 이용하여 데이터 분석 및 중고차 가격 예측을 진행합니다.

<br><br>

---

<br><br>

## **1. 데이터 불러오기**

<br>

우선, 데이터를 분석하기 위하여 주어진 train 데이터셋과 test 데이터셋을 읽어와 저장하고, 데이터 구조를 확인합니다.


In [2]:
import pandas as pd ## pandas 라이브러리를 가져오기 위하여 import를 해줍니다.

# csv 형식으로 된 데이터 파일을 읽어옵니다.
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')

print(f'train data set은 {train.shape[1]} 개의 feature를 가진 {train.shape[0]} 개의 데이터 샘플로 이루어져 있습니다.')

# 데이터의 최상단 5 줄을 표시합니다.
train.head()

train data set은 11 개의 feature를 가진 1015 개의 데이터 샘플로 이루어져 있습니다.


,id,title,odometer,location,isimported,engine,transmission,fuel,paint,year,target
0,0,Toyota RAV 4,18277,Lagos,Foreign Used,4-cylinder(I4),automatic,petrol,Red,2016,13665000
1,1,Toyota Land Cruiser,10,Lagos,New,4-cylinder(I4),automatic,petrol,Black,2019,33015000
2,2,Land Rover Range Rover Evoque,83091,Lagos,Foreign Used,6-cylinder(V6),automatic,petrol,Red,2012,9915000
3,3,Lexus ES 350,91524,Lagos,Foreign Used,4-cylinder(I4),automatic,petrol,Gray,2007,3815000
4,4,Toyota Venza,94177,Lagos,Foreign Used,6-cylinder(V6),automatic,petrol,Red,2010,7385000


In [ ]:
print(train.shape, test.shape)

(1015, 11) (436, 10)


csv 파일을 읽어와 데이터프레임으로 저장 및 출력, 가공에 필요한 라이브러리인 pandas, numpy, matplotlib를 import하였습니다.<br>

데이콘에서 제시한 훈련용 파일인 train.csv와 테스트용 파일인 test.csv를 read_csv()함수를 이용하여 읽어와 데이터프레임으로 변환하여, 각각 train과 test 변수에 저장하였습니다.<br>

shape 함수를 이용하여 해당 데이터 셋의 행 및 열의 개수를 확인할 수 있습니다.<br>

head() 함수를 이용하여 데이터의 첫 다섯 줄을 출력하여 데이터 셋의 전체적인 구성을 확인합니다.<br>

주어진 데이터에는 중고차에 관한 id(샘플 아이디), title(제조사 모델명), odmeter(주행 거리), location(판매처), isimported(현지 사용 여부), engine(엔진 종류), transmission(트랜스미션 종류), fuel(연료 종류), paint(색상), year(제조년도), target(자동차 가격)이라는 11개의 컬럼이 존재하는 것을 알 수 있습니다.<br>
그 중에서 title, location, isimported, engine, transmission, fuel, paint 변수는 문자열 형태인 범주형 변수인 것을 확인할 수 있습니다. 머신러닝 등의 기법을 사용하여 분석할 경우, 전처리가 필요할 것으로 예상됩니다.<br>

train data는 총 11개의 컬럼과 1015개의 (row)데이터로 이루어져있음을 알 수 있습니다.<br>
test data는 target 변수를 제외한 10개의 컬럼과 436개의 데이터로 이루어져있는 것을 알 수 있습니다.

<br><br>

## **2. 결측치 확인**

<br>

결측치란, Missing Value 혹은 NA라고도 표현하며, 말그대로 값이 존재하지 않는 데이터를 의미합니다.<br>

이러한 결측치가 train data에 포함되어 있다면 보다 정확한 분석을 하기 어렵습니다. 따라서, 데이터에 결측치가 포함되어있는지 확인하고 제거하거나 적절히 대체해주어야 합니다.

<br>

함수를 작성하여 데이터셋에 결측치가 존재하는지 확인합니다.

In [3]:
def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col} 입니다')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(train)

결측치가 존재하지 않습니다


isna() 함수를 통해 결측치를 체크하는 check_missing_col() 함수를 작성하여 train 데이터 셋에 결측치가 포함되어있는지 확인합니다.<br>

데이터셋에 결측치가 존재하지 않는 것을 확인하였습니다. 따라서 결측치 처리는 생략합니다.

<br><br>

## **3. 데이터 전처리**

<br>

데이터 전처리란, 데이터를 분석하기에 알맞게 가공하여 처리하는 과정으로 데이터 분석 과정에서 반드시 거쳐야 하는 과정입니다.<br>

데이터에 존재하는 결측값, 노이즈, 이상값 등을 제거/대체하고 범주형 변수를 데이터 분석에 알맞게 변환하는 작업을 진행합니다.<br>
결측치는 존재하지 않는 것이 이전 단계에서 확인되었으므로 결측치 처리는 생략합니다.

<br>

오타 처리, 파생변수 생성, 데이터 정제 등의 데이터 전처리를 시행합니다.

<br>

### 3-1. 오탈자 처리

<br>

In [4]:
train[train['year'] < 1900]

,id,title,odometer,location,isimported,engine,transmission,fuel,paint,year,target
415,415,Mercedes-Benz ATEGO,403461,Lagos,Locally used,4-cylinder(I4),manual,diesel,white,1218,6015000
827,827,Mercedes-Benz/52,510053,Lagos,Locally used,6-cylinder(V6),manual,diesel,white,1217,75015000


year 칼럼에 잘못 기입된 것으로 보이는 1218, 1217년도의 데이터가 존재하는 것을 확인할 수 있습니다.

오기입된 데이터는 제거합니다.

In [ ]:
train = train[train['year'] > 1900]
train = train.drop('id', axis = 1).reset_index().drop('index', axis = 1).reset_index().rename({'index':'id'}, axis = 'columns')
train.shape

(1013, 11)

오기입된 year 데이터를 포함한 2개의 데이터를 제거하여 1013개의 데이터가 남은 것을 확인할 수 있습니다.

<br>

### 3-2. 파생변수 생성

<br>

데이터에 존재하는 카테고리들을 같은 카테고리로 뭉쳐 파생변수를 만듭니다.<br>

파생변수란, 기존의 변수들을 조합하여 만든 새로운 변수로 합계, 평균 등이 속합니다.

<br>

데이터에 존재하는 카테고리들을 같은 카테고리로 뭉쳐 파생변수를 만듭니다.<br>
파생변수란, 기존의 변수들을 조합하여 만든 새로운 변수로 합계, 평균 등이 속합니다.<br>

우선, 카테고리 개수가 많은 title 변수를 대상으로 파생변수를 만들어 고유 값을 유의하게 줄여줍니다. 

In [ ]:
train['title'].value_counts()[:20]

Toyota Camry            129
Toyota Corolla          108
Toyota Highlander        50
Toyota Sienna            49
Toyota RAV 4             40
Honda Accord             38
Lexus RX 350             33
Lexus ES 350             24
Toyota Venza             16
Lexus GX 460             16
Honda Pilot              15
Lexus LX 570             14
Mercedes-Benz ML 350     13
Toyota Avalon            13
Mercedes-Benz GL 450     12
Toyota Matrix            12
Toyota Tacoma            12
Ford Explorer            12
Mercedes-Benz E 350      12
Toyota 4-Runner          12
Name: title, dtype: int64

제조사 모델명을 뜻하는 title 변수의 카테고리를 살펴보면 Toyota Camry, Toyota Corolla, Toyota Hiighlander 등 중복된 키워드가 다수 포함되어 있는 것을 확인할 수 있습니다.<br>
중고차 가격 분석 시, 굳이 차종을 전부 따지지 않고 Toyota, Lexus 등의 브랜드명으로 묶어 분석하는 편이 시각화 및 머신러닝에 도움이 될 것 같습니다.<br>

Title이라는 카테고리를 상위 변수로 두어 브랜드 별 카테고리 변수를 추가하여 분석을 더 용이하게 하게 해줍니다.<br>

In [ ]:
train['brand'] = train['title'].apply(lambda x : x.split(" ")[0])
train.head()

,id,title,odometer,location,isimported,engine,transmission,fuel,paint,year,target,brand
0,0,Toyota RAV 4,18277,Lagos,Foreign Used,4-cylinder(I4),automatic,petrol,Red,2016,13665000,Toyota
1,1,Toyota Land Cruiser,10,Lagos,New,4-cylinder(I4),automatic,petrol,Black,2019,33015000,Toyota
2,2,Land Rover Range Rover Evoque,83091,Lagos,Foreign Used,6-cylinder(V6),automatic,petrol,Red,2012,9915000,Land
3,3,Lexus ES 350,91524,Lagos,Foreign Used,4-cylinder(I4),automatic,petrol,Gray,2007,3815000,Lexus
4,4,Toyota Venza,94177,Lagos,Foreign Used,6-cylinder(V6),automatic,petrol,Red,2010,7385000,Toyota


Toyota Camry의 Toyota처럼, 브랜드명은 띄어쓰기로 구분되어 첫 번째 단어로 포함되어 있으므로 title 변수 데이터를 split하여 첫 번째(0번째 리스트) 문자열을 추출합니다.<br>
추출해낸 문자열 데이터를 'brand'라는 칼럼으로 새로 생성하여 추가합니다. 이로써 title의 첫 번째 단어를 이용한 파생 변수 칼럼이 생성되었습니다.

<br><br>

### 3-3. 데이터 정제

<br>

효과적인 데이터 분석을 위해서는 주어진 데이터 셋의 포맷을 통일하고, 불필요한 값을 제거하거나 통합하고, 오탈자를 수정하는 데이터 정제 과정을 거쳐야 합니다.

<br>

우선, paint 컬럼의 카테고리를 출력해봅니다.

In [ ]:
train['paint'].value_counts()[:20]

Black        292
White        115
Silver       103
Grey          84
Blue          70
Red           53
Dark Grey     47
Gold          37
white         24
Green         18
Gray          17
Brown         16
Black         15
Dark Blue     14
Wine           8
Silver         5
Sliver         5
blue           5
red            5
Grey           3
Name: paint, dtype: int64

Black과 black, Red와 red 등 대소문자 표기가 일치하지 않아 다른 카테고리로 나뉘어진 경우가 눈에 띕니다. 또한, Silver의 오타인 Sliver 카테고리도 보입니다.

이들의 표기형식을 통일하고 오타 및 표기를 수정합니다.

우선, 정규표현식을 이용하여 공백 및 대소문자 표기를 통일시켜줍니다.



In [ ]:
import re 

def clean_text(texts): 
    corpus = [] 
    for i in range(0, len(texts)): 
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"\n\]\[\>\<]', '',texts[i]) #@%*=()/+ 와 같은 문장부호 제거
        review = re.sub(r'\d+','',review)#숫자 제거
        review = review.lower() #소문자 변환
        review = re.sub(r'\s+', ' ', review) #extra space 제거
        review = re.sub(r'<[^>]+>','',review) #Html tags 제거
        review = re.sub(r'\s+', ' ', review) #spaces 제거
        review = re.sub(r"^\s+", '', review) #space from start 제거
        review = re.sub(r'\s+$', '', review) #space from the end 제거
        review = re.sub(r'_', ' ', review) #space from the end 제거
        #review = re.sub(r'l', '', review)
        corpus.append(review) 
        
    return corpus

정규표현식을 다루는 re 모듈의 sub 메소드를 이용하여 데이터들을 수정합니다.<br>

입력받은 값의 숫자를 제거하고, 대소문자 통일을 위하여 모든 알파벳을 소문자로 변환하고, 공백 및 띄어쓰기를 제거하는 clear_text 함수를 작성하였습니다.

In [ ]:
print('정제 전 brand의 unique 카테고리 개수 : ', len(train['paint'].unique()))

temp = clean_text(train['paint']) #메소드 적용
train['paint'] = temp

print('정제 후 brand의 unique 카테고리 개수 : ', len(train['paint'].unique()))

정제 전 brand의 unique 카테고리 개수 :  76
정제 후 brand의 unique 카테고리 개수 :  51


작성한 clean_text 함수를 train 데이터 셋의 paint 칼럼에 적용합니다.<br>
이로써 대소문자 구분이나 공백 문자열 때문에 다른 데이터로 나뉘어졌던 데이터들을 하나로 합쳐주었습니다.

76개의 카테고리가 존재했던 paint 변수의 카테고리가 51개로 줄어든 모습입니다.

In [ ]:
train['paint'].value_counts()[:20]

black        310
white        142
silver       109
grey          87
blue          78
red           58
dark grey     48
gold          41
green         20
gray          18
dark blue     17
brown         16
wine           9
sliver         5
dark ash       4
cream          4
ash            3
purple         3
yellow         3
whiteblue      2
Name: paint, dtype: int64

띄어쓰기와 대소문자 문제는 해결하였으나, 오탈자와 색상의 세세한 구분은 여전히 존재합니다.<br>

오타를 수정하고 색상의 명도/채도 등 적은 차이의 색상 표현(dark grey와 grey 등)은 해당 색상을 대표할 수 있는 한 카테고리로 통합해주면 카테고리를 더욱 줄일 수 있을 것 같습니다.

In [ ]:
train['paint'] = train['paint'] = train['paint'].apply(lambda x : 'blue' if x.find('blue') >= 0 else x)
train['paint'] = train['paint'] = train['paint'].apply(lambda x : 'red' if x.find('red') >= 0 else x)
train['paint'] = train['paint'] = train['paint'].apply(lambda x : 'green' if x.find('green') >= 0 else x)
train['paint'] = train['paint'] = train['paint'].apply(lambda x : 'white' if x.find('white') >= 0 else x)
train['paint'] = train['paint'] = train['paint'].apply(lambda x : 'grey' if x.find('grey') >= 0 else x)
train['paint'] = train['paint'] = train['paint'].apply(lambda x : 'grey' if x.find('gery') >= 0 else x)
train['paint'] = train['paint'] = train['paint'].apply(lambda x : 'grey' if x.find('gray') >= 0 else x)
train['paint'] = train['paint'] = train['paint'].apply(lambda x : 'ash' if x.find('ash') >= 0 else x)
train['paint'] = train['paint'] = train['paint'].apply(lambda x : 'brown' if x.find('brown') >= 0 else x)
train['paint'] = train['paint'] = train['paint'].apply(lambda x : 'silver' if x.find('silver') >= 0 else x)
train['paint'] = train['paint'] = train['paint'].apply(lambda x : 'silver' if x.find('sliver') >= 0 else x)
train['paint'] = train['paint'] = train['paint'].apply(lambda x : 'black' if x.find('black') >= 0 else x)
train['paint'] = train['paint'] = train['paint'].apply(lambda x : 'gold' if x.find('gold') >= 0 else x)
train['paint'] = train['paint'] = train['paint'].apply(lambda x : 'wine' if x.find('whine') >= 0 else x)

sliver라는 오타 문자열이 확인되면 silver로 변환하여 주고, grey라는 문자열이 포함된 dark grey가 확인되면 grey로 변환하여 대표 색상으로 통일해주었습니다.

In [ ]:
train['paint'].value_counts()

black     312
grey      159
white     146
silver    117
blue      105
red        60
gold       42
green      21
brown      17
wine       10
ash         8
cream       4
purple      3
yellow      3
milk        2
maroon      2
beige       1
orange      1
Name: paint, dtype: int64

In [ ]:
print('paint의 unique 카테고리 개수 : ', len(train['paint'].value_counts()))

paint의 unique 카테고리 개수 :  18


데이터 정제 처리를 통해, 처음에는 76개였던 paint 변수의 카테고리를 데이터 분석에 용이하도록 18개까지 줄여주었습니다.

<br>

### 3-4. 라벨 인코딩

<br>

데이터셋의 id, odometer, year, target 칼럼만이 int형이고, 나머지 칼럼들은 object 데이터형입니다.

데이터 분석에 사용하고자 하는 XGBoost 머신러닝 기법은 범주형 데이터를 이용할 수 없으므로, 범주형 변수의 경우 전처리를 통해 정수 값으로 변환해주어야 합니다.

Label Encoding 기법을 이용하여 범주형 변수의 문자열을 수치형, 즉 숫자 값으로 변환해주도록 합니다.

In [ ]:
#라벨인코딩을 하기 위함 dictionary map 생성 함수
def make_label_map(dataframe):
    label_maps = {}
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            label_map = {'unknown':0}
            for i, key in enumerate(dataframe[col].unique()):
                label_map[key] = i+1  #새로 등장하는 유니크 값들에 대해 1부터 1씩 증가시켜 키값을 부여해줍니다.
            label_maps[col] = label_map
    print(label_maps)
    return label_maps

# 각 범주형 변수에 인코딩 값을 부여하는 함수
def label_encoder(dataframe, label_map):
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            dataframe[col] = dataframe[col].map(label_map[col])
            dataframe[col] = dataframe[col].fillna(label_map[col]['unknown']) #혹시 모를 결측값은 unknown의 값(0)으로 채워줍니다.
    return dataframe

le = make_label_map(train)
train = label_encoder(train, le)
train.head()

{'title': {'unknown': 0, 'Toyota RAV 4': 1, 'Toyota Land Cruiser': 2, 'Land Rover Range Rover Evoque': 3, 'Lexus ES 350': 4, 'Toyota Venza': 5, 'Toyota Corolla': 6, 'Land Rover Range Rover Sport': 7, 'Pontiac Vibe': 8, 'Toyota Tacoma': 9, 'Lexus RX 350': 10, 'Ford Escape': 11, 'Honda Civic': 12, 'Volvo XC90': 13, 'BMW 750': 14, 'Infiniti JX': 15, 'Honda Accord': 16, 'Mercedes-Benz ML 350': 17, 'Toyota Camry': 18, 'Hyundai Azera': 19, 'Lexus GX 460': 20, 'BMW 325': 21, 'Toyota Sienna': 22, 'Honda Fit': 23, 'Honda CR-V': 24, 'Hyundai Tucson': 25, 'Ford Transit': 26, 'Jaguar XJ8': 27, 'BMW X6': 28, 'Mercedes-Benz C 300': 29, 'Mitsubishi Galant': 30, 'Mercedes-Benz GL 450': 31, 'Lexus RX 300': 32, 'Toyota Highlander': 33, 'Mitsubishi CANTER PICK UP': 34, 'Nissan Titan': 35, 'Lexus IS 250': 36, 'Mercedes-Benz 200': 37, 'Toyota Sequoia': 38, 'Ford Explorer': 39, 'Hyundai ix35': 40, 'Mack CH613 CONVENTIONAL CAB': 41, 'Lexus CT 200h': 42, 'Lexus LX 570': 43, 'Toyota Avensis': 44, 'Toyota 4-Run

,id,title,odometer,location,isimported,engine,transmission,fuel,paint,year,target,brand
0,0,1,18277,1,1,1,1,1,1,2016,13665000,1
1,1,2,10,2,2,1,1,1,2,2019,33015000,1
2,2,3,83091,2,1,2,1,1,1,2012,9915000,2
3,3,4,91524,2,1,1,1,1,3,2007,3815000,3
4,4,5,94177,2,1,2,1,1,1,2010,7385000,1


우선, 라벨 인코딩을 하기 위해 dictionary map을 생성하는 함수 make_label_map 함수와, make_label_map에서 반환한 유니크 값을 이용하여 범주형 변수에 인코딩 값을 부여하는 label_encoder 함수를 작성하고 적용하였습니다.<br>

범주형 변수들을 문자열이 아닌 숫자 형태로 변환해준 모습입니다.<br>

각 범주 값에 1에서 N(총 카테고리)까지의 숫자가 부여되었습니다.<br>
예를 들어 paint에서 Red는 1로, Black은 2로, Gray는 3으로 변환되었습니다.

<br><br>

## **4. 모델링**

### 4-1. 변수 정의

<br>

해당 데이터 분석의 목적은 주어진 중고차 상세 정보 데이터를 이용하여 target 변수인 자동차 가격을 예측하는 것입니다. <br>

이를 바탕으로 독립변수 X는 중고차 상세 정보 데이터, 종속변수 y는 중고차 가격인 target 변수가 될 것입니다.

In [ ]:
X = train.drop(['id', 'target'], axis = 1) #training 데이터에서 피쳐 추출
y = train.target #training 데이터에서 중고차 가격 추출

기본적인 인덱스를 의미하는 id 칼럼과 종속변수가 될 target 변수를 제외한 나머지 데이터를 독립변수 X로 설정하였습니다.

그리고 중고차 가격인 target 칼럼만 추출하여 종속변수로 설정하였습니다.

<br>

### 4-2. train/validation 정의

<br>

주어진 데이터 중 train 데이터셋을 모두 이용하여 학습시킨 뒤, 바로 test 데이터셋에 넣어 예측하면 train 데이터셋으로 학습시킨 모델이 제대로 만들어졌는지, 즉 유의한지 확인할 수 없습니다. <br>

따라서 train 데이터셋의 일부는 모델을 학습하는 데 사용하고, train의 나머지 데이터셋은 모델이 유의한지 확인하는 용도로 사용합니다.<br>
높은 정확도가 출력되어 유의하다는 것이 확인되었을 때 train 데이터셋을 전부 이용하여 학습하고 test 데이터셋에 적용하도록 합니다.

<br>

train 데이터 셋을 각각 75%와 25%로 나누어, 학습시키는 용도로 사용될 train_data 데이터셋과 모델이 유의한지 검증을 위한 val_dat 데이터셋으로 나누어줍니다.

In [ ]:
from sklearn.model_selection import train_test_split

data = train.drop('id', axis = 1).copy() #필요없는 id열 삭제
train_data, val_data = train_test_split(data, test_size=0.25) # 25프로로 설정
train_data.reset_index(inplace=True) #전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화
val_data.reset_index(inplace=True)

print('학습시킬 train 셋 : ', train_data.shape)
print('검증할 val 셋 : ', val_data.shape)

학습시킬 train 셋 :  (759, 12)
검증할 val 셋 :  (254, 12)


기본적인 인덱스를 나타내는 id 칼럼을 제거한 후, sklearn의 train_test_split 메소드를 이용하여 75%의 학습용 데이터셋과 25%의 검증용 데이터셋으로 나눠주었습니다.

train 데이터셋(총 761개의 데이터)으로 모델을 학습시키고 val 데이터셋(총 254개의 데이터)으로 모델의 성능이 유의한지 검증하도록 합니다.

In [ ]:
train_data_X = train_data.drop(['target', 'index'], axis = 1) #training 데이터에서 피쳐 추출
train_data_y = train_data.target #training 데이터에서 target 추출

val_data_X = val_data.drop(['target', 'index'], axis = 1) #training 데이터에서 피쳐 추출
val_data_y = val_data.target #validation 데이터에서 target 추출

나뉜 train 데이터 셋에서 target변수와 일련번호를 뜻하는 index 칼럼을 제외하여 train_data_X로 정의하고 target 변수만 추출하여 train_data_y로 정의하였습니다.<br>

val 데이터 역시 target변수와 일련번호를 뜻하는 index 칼럼을 제외하여 val_data_X로 정의하고 target 변수만 추출하여 val_data_y로 정의하였습니다.

<br><br>

## **5. XGBoost**

### 5-1. XGBoost의 정의

<br>

우선 Boosting이란, 여러 개의 약한 의사결정나무를 조합해서 사용하는 앙상블 기법 중 하나입니다. 즉, 약한 예측 모형들을 통해 가중치를 정하고, 순차적으로 다음 학습 모델에 반영하여 강한 예측 모형을 만들어나가는 것입니다.

<br>

XGBoost란 Extreme Gradient Boosting의 약자로, Boosting 기법을 이용해 구현한 알고리즘인 Gradient Boost를 병렬 학습이 지원되도록 구현한 라이브러리입니다.<br>

Regression, Classfication 문제를 모두 지원하며, 성능과 자원 효율이 좋아 많이 사용되고 있는 알고리즘입니다.

<br>

### 5-2. XGBoost의 하이퍼파라미터

<br>

XGBoost에는 다수의 하이퍼파라미터가 존재하며, 일반/부스터/학습 과정 파라미터로 나뉩니다.
- 일반 파라미터 : 부스팅을 수행할 때 트리를 사용할지, 선형 모델을 사용할지 등을 선택합니다.
- 부스터 파라미터 : 선택한 부스터에 따라 적용할 수 있는 파라미터 종류가 다릅니다.
- 학습 과정 파라미터 : 학습 시나리오를 결정합니다.

<br>

> 일반 파라미터
- booster : 어떤 부스터 구조를 쓸지 결정합니다. 의사결정기반모형(gbtree), 선형모형(gblinear), dart가 있습니다.
- n_jobs : 사용되는 병렬 스레드 수를 결정합니다.
- verbosity : 0(무음), 1(경고), 2(정보), 3(디버그)를 의미합니다.

> 부스터 파라미터
- learning_rate : 기본값은 0.3이며 높을수록 과적합되기 쉽습니다.
- n_estimators : 생성할 weak learner의 수이며 기본값은 100입니다. learning_rate가 낮을 때에는 n_estimators를 높여야 과적합이 방지됩니다.
- max_depth : 적절한 값이 제시되어야 하며 보통 3~10 사이 값이 적용됩니다. 높아질수록 모델의 복잡도가 커져 과적합되기 쉽습니다. 기본값은 6.
- min_child_weight : 관측치에 대한 가중치 합의 최소입니다. 높아질수록 과적합이 방지됩니다. 기본값은 1.
- subsample : 학습에 사용하는 데이터 샘플링 비율입니다. 보통 0.5~1 정도가 사용되며, 값이 낮을수록 과적합이 방지됩니다.
- colsample_bytree : 각 트리 별 사용된 feature의 퍼센테이지입니다. 보통 0.5~1 정도로 사용됩니다. 값이 낮을수록 과적합이 방지됩니다.

> 학습 과정 파라미터
- objective : reg:squarederror는 제곱 손실이 있는 회귀를 뜻하며 기본 값입니다. binary:logistic은 이항 분류 문제 로지스틱 회귀 모형을 예측 확률을 반환합니다. multi:softmax는 다항 분류 문제에 사용됩니다. multi:softprob는 각 클래스 범주에 속하는 예측확률을 반환합니다.
- eval_metric : 모델의 평가 함수를 조정하는 함수입니다. rmse, mae, logloss 등으로 설정할 수 있습니다.
- seed : 재현 가능하도록 난수를 고정시킵니다.

<br>

과적합 방지를 위해서는 learning rate, max_depth 등을 낮추고 n_estimators, min_child_weight, gamma 등은 높이는 조정을 할 수 있습니다.

<br><br>

## **6. 모델링**

### 6-1. 모델링 및 하이퍼파라미터 튜닝

<br>

일반적으로 하이퍼파라미터 튜닝은
1. high learning rate(0.05 - 0.3)를 선택하고 이 학습률에 맞는 tree 개수를 선정한다.
2. tree-specific parameter를 수정한다.
3. regularization parameter를 수정한다.
4. 학습률을 낮추고 다시 반복한다.

의 순서로 진행됩니다.

<br>

우선, Learning rate 와 estimator의 수를 설정합니다.<br>

초기값으로는 각 파라미터마다 적정값에 따라 max_depth는 4~6, min_child_weight는 1, subsample는 0.9, colsample_bytree는 0.8, scale_pos_weight는 1로 시작하여 튜닝해나갈 것입니다.

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                           colsample_bytree=0.8, 
                           learning_rate=0.3, 
                           max_depth=6, 
                           n_estimators=100, 
                           subsample=0.9, 
                           min_child_weight=1,
                          scale_pos_weight=1,
                           seed=1)

objective(목적 함수)는 기본 값을 설정해주었고, seed 값은 결괏값 고정을 위해 1로 지정해주었습니다.

파라미터들은 우선 위처럼 적정값의 평균 정도에 맞추어 설정한 뒤, XGBRegressor()를 이용해 XGBoost 회귀 모델로 모델을 정의하였습니다.

In [ ]:
xgb_reg.fit(train_data_X, train_data_y, eval_set=[(train_data_X, train_data_y), (val_data_X, val_data_y)], early_stopping_rounds=300, verbose=False)

XGBRegressor(colsample_bytree=0.86, eta=0.01, learning_rate=0.3, max_depth=6,
             objective='reg:squarederror', seed=1, subsample=0.9)

fit()을 이용하여 train 데이터로 모델을 학습하였습니다. eval_set은 val 데이터로 지정하여, 검증 세트를 val 데이터로 하여 예측 오류값을 줄이고 오버 피팅은 줄일 수 있도록 하였습니다.

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

def nmae(true, pred):

    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    
    return score

y_hat = xgb_reg.predict(val_data_X) # y예측
print(f'모델 NMAE: {nmae(val_data_y,y_hat)}')

모델 NMAE: 0.35751279308806594


예측한 값을 NMAE 평가산식으로 출력하자, 0.3575 정도의 값이 나온 모습입니다. <br>

하이퍼파라미퍼 튜닝을 하지 않았을 때의 XGBoost를 이용한 예측 결과의 오차값은 35.7% 정도로, 의사결정나무나 앙상블 기법을 사용했을 때보다 약간 낮은 오차값을 보이는 모습입니다.

<br>

오차값을 줄이고 정확도를 높이기 위하여 하이퍼파라미터를 튜닝합니다.

우선, learning_rate와 n_estimators를 조정하여 오차를 줄입니다.

learning_rate는 낮은 값일수록 모델이 견고해지고 높을수록 과적합의 위험이 있습니다.<br>
n_estimators는 높을수록 과적합을 예방할 수 있습니다.<br>

두 파라미터를 조정해가며 최적의 오차값이 나오는 값을 찾고, 최종적으로 learning_rate는 0.000997, n_estimators는 60000로 설정해주었습니다.

In [ ]:
xgb_reg = xgb.XGBRegressor(
    objective ='reg:squarederror', 
    colsample_bytree=0.8, 
    learning_rate=0.000997, 
    max_depth=6, 
    n_estimators=60000, 
    subsample=0.9, 
    min_child_weight=1,
    scale_pos_weight=1,
    seed=1)
xgb_reg.fit(train_data_X, train_data_y, eval_set=[(train_data_X, train_data_y), (val_data_X, val_data_y)], early_stopping_rounds=300, verbose=False)
y_hat = xgb_reg.predict(val_data_X) # y예측
print(f'모델 NMAE: {nmae(val_data_y,y_hat)}')

모델 NMAE: 0.3289287196312952


예측한 값을 NMAE 평가산식으로 출력하자, 0.3289 정도의 값이 나온 모습입니다. <br>

오차값은 33% 정도로, 튜닝 전보다 줄어들은 모습입니다.

<br>

다음으로는 max_depth와 min_child_weight 하이퍼파라미터를 조정합니다.

max_depth는 값이 높아질수록 과적합 될 위험이 있습니다. 통상적인 범위(3~10) 내에서 오차값이 충분히 줄어들 정도로 높여서 10으로 설정하였습니다.

min_child_weight는 높아질수록 과적합이 방지됩니다. 오차값이 크게 상승하지 않는 한에서 높여서 3으로 설정하였습니다.

In [ ]:
xgb_reg = xgb.XGBRegressor(
    objective ='reg:squarederror', 
    colsample_bytree=0.8, 
    learning_rate=0.000997, 
    max_depth=10, 
    n_estimators=60000, 
    subsample=0.9, 
    min_child_weight=3,
    scale_pos_weight=1,
    seed=1)
xgb_reg.fit(train_data_X, train_data_y, eval_set=[(train_data_X, train_data_y), (val_data_X, val_data_y)], early_stopping_rounds=300, verbose=False)
y_hat = xgb_reg.predict(val_data_X) # y예측
print(f'모델 NMAE: {nmae(val_data_y,y_hat)}')

모델 NMAE: 0.306278773499749


예측한 값을 NMAE 평가산식으로 출력하자, 0.3062 정도의 값이 나온 모습입니다. <br>

오차값은 30.6% 정도로, max_depth와 min_child_weight를 튜닝하기 전보다 줄어든 모습입니다.

<br>

다음으로는 colsample_bytree와 subsample 하이퍼파라미터를 조정합니다.

두 파라미터 모두 범위값(0~1) 이내에서 오차값이 줄어들도록 조정해주었습니다.

최종적으로 colsample_bytree는 0.8, subsample은 0.94로 설정하였습니다.

In [ ]:
xgb_reg = xgb.XGBRegressor(
    objective ='reg:squarederror', 
    colsample_bytree=0.8, 
    learning_rate=0.000997, 
    max_depth=10, 
    n_estimators=60000, 
    subsample=0.94, 
    min_child_weight=3,
    scale_pos_weight=1,
    seed=1)
xgb_reg.fit(train_data_X, train_data_y, eval_set=[(train_data_X, train_data_y), (val_data_X, val_data_y)], early_stopping_rounds=300, verbose=False)
y_hat = xgb_reg.predict(val_data_X) # y예측
print(f'모델 NMAE: {nmae(val_data_y,y_hat)}')

모델 NMAE: 0.30484334905127464


예측한 값을 NMAE 평가산식으로 출력하자, 0.3062 정도의 값이 나온 모습입니다. <br>

오차값은 30.4% 정도로, XGBoost의 하이퍼파라미터를 튜닝하기 전보다 약 5.3% 정도 감소한 것을 확인할 수 있습니다.

<br><br>

## **. train 데이터셋 + val 데이터셋 100% 활용하는 모델 만들기**

<br>

학습한 모델의 성능이 어느 정도인지 파악하였으니, 나눠두었던 train 데이터셋과 val 데이터셋을 100% 모두 이용하여 모델을 학습시킵니다.

In [ ]:
train_X = train.drop(['id', 'target'], axis = 1) #training 데이터에서 피쳐 추출
train_y = train.target #training 데이터에서 target 추출

In [ ]:
xgb_reg.fit(train_X, train_y, eval_set=[(train_X, train_y)], early_stopping_rounds=300, verbose=False)

XGBRegressor(colsample_bytree=0.8, eta=0.01, learning_rate=0.000997,
             max_depth=10, min_child_weight=3, n_estimators=60000,
             objective='reg:squarederror', seed=1, subsample=0.5)

<br><br>

## **. 예측하기**

<br>

학습한 모델을 이용하여 test 데이터셋을 예측해봅니다.

변수의 형태 및 개수를 동일하게 해주기 위해, test 데이터셋에도 앞서 train 데이터셋에 진행하였던 전처리 과정을 동일하게 진행해줍니다.

In [ ]:
check_missing_col(test) # 결측치 확인

test = test.drop('id', axis = 1) #분석에 필요없는 열 삭제

test['brand'] = test['title'].apply(lambda x : x.split(" ")[0])

temp = clean_text(test['paint']) #메소드 적용
test['paint'] = temp
test['paint'] = test['paint'] = test['paint'].apply(lambda x : 'blue' if x.find('blue') >= 0 else x)
test['paint'] = test['paint'] = test['paint'].apply(lambda x : 'red' if x.find('red') >= 0 else x)
test['paint'] = test['paint'] = test['paint'].apply(lambda x : 'green' if x.find('green') >= 0 else x)
test['paint'] = test['paint'] = test['paint'].apply(lambda x : 'white' if x.find('white') >= 0 else x)
test['paint'] = test['paint'] = test['paint'].apply(lambda x : 'grey' if x.find('grey') >= 0 else x)
test['paint'] = test['paint'] = test['paint'].apply(lambda x : 'grey' if x.find('gery') >= 0 else x)
test['paint'] = test['paint'] = test['paint'].apply(lambda x : 'grey' if x.find('gray') >= 0 else x)
test['paint'] = test['paint'] = test['paint'].apply(lambda x : 'ash' if x.find('ash') >= 0 else x)
test['paint'] = test['paint'] = test['paint'].apply(lambda x : 'brown' if x.find('brown') >= 0 else x)
test['paint'] = test['paint'] = test['paint'].apply(lambda x : 'silver' if x.find('silver') >= 0 else x)
test['paint'] = test['paint'] = test['paint'].apply(lambda x : 'silver' if x.find('sliver') >= 0 else x)
test['paint'] = test['paint'] = test['paint'].apply(lambda x : 'black' if x.find('black') >= 0 else x)
test['paint'] = test['paint'] = test['paint'].apply(lambda x : 'gold' if x.find('gold') >= 0 else x)
test['paint'] = test['paint'] = test['paint'].apply(lambda x : 'wine' if x.find('whine') >= 0 else x)

test = label_encoder(test, le)

결측치가 존재하지 않습니다


전처리를 완료한 test 데이터셋을 이용하여 앞서 만들어두었던 XGBoost 모델로 예측합니다.

In [ ]:
y_pred = xgb_reg.predict(test)
y_pred[0:5]

array([12399618. ,  4556846.5,  6437247. ,   761937.8,  2391824.2],
      dtype=float32)

<br><br>

## **9. 파일 저장 및 제출**

<br>

학습한 모델을 이용한 예측 결과를 csv 파일로 제작하여 제출합니다.

In [ ]:
# 제출용 sample 파일을 불러옵니다.
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission.csv')
submission.head()

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


제출용으로 제시된 sample 파일을 pandas 모듈의 read_csv를 통해 읽어옵니다.

In [ ]:
# 위에서 구한 예측값을 그대로 넣어줍니다.
submission['target'] = y_pred
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/XGBoost3.csv', index=False)

# 데이터가 잘 들어갔는지 확인합니다.
submission.head()

,id,target
0,0,1.239962e+07
1,1,4.556846e+06
2,2,6.437247e+06
3,3,7.619378e+05
4,4,2.391824e+06


앞서 모델을 이용해 예측했던 데이터 값들을 sample 데이터프레임의 target 칼럼에 넣어줍니다.

예측값을 넣은 데이터프레임을 to_csv()를 이용하여 csv 파일로 저장합니다. 이때, index=False를 지정하여 추가적인 id를 지정하지 않도록 합니다.

<br><br>

## **출처**

<br>

베이스라인 코드 : 
- https://dacon.io/competitions/official/235901/codeshare/5103?page=1&dtype=recent (데이콘)
- +) 박연호 동기님이 공유해주신 코드

<br>

용어 및 라이브러리/모듈 설명 : 
- https://www.kaggle.com/code/lifesailor/xgboost/notebook
- https://psystat.tistory.com/131#head1
- https://dining-developer.tistory.com/4